In [15]:
import rdflib
from rdflib.plugins.sparql import prepareQuery
from tabulate import tabulate

In [16]:
filename = "ABox.ttl"

In [17]:
text1 = '''CQ_7.1
What are the steps that are part of the workflow for the risk assessment of the museum? What is their duration? What are the activities they were executed in?
'''

query1 = '''
PREFIX tbox: <https://w3id.org/sirius/ontology/development/07/schema/>
PREFIX abox: <https://w3id.org/sirius/ontology/development/07/data/>

SELECT DISTINCT ?step ?duration_value ?activity
WHERE {
  ?workflow tbox:hasStep ?step .
  ?step tbox:hasDuration ?duration ;
    tbox:isExecutedIn ?activity .
  ?duration tbox:hasDays ?duration_value .
}
'''

In [18]:
text2 = '''CQ_7.2
What are the activities involved in the event executing the workflow? What are the time interval in which they respectively are executed?
'''

query2 = '''
PREFIX tbox: <https://w3id.org/sirius/ontology/development/07/schema/>
PREFIX abox: <https://w3id.org/sirius/ontology/development/07/data/>

SELECT DISTINCT ?activity ?start_date ?end_date
WHERE {
    ?workflow_execution tbox:involvesActivity ?activity .
    ?activity tbox:atTime ?time_interval .
    ?time_interval tbox:hasStartDate ?start_date ;
      tbox:hasEndDate ?end_date .
}
'''

In [19]:
text3 = '''CQ_7.3
What does each assessment activity target? What does it assess?
'''

query3 = '''
PREFIX tbox: <https://w3id.org/sirius/ontology/development/07/schema/>
PREFIX abox: <https://w3id.org/sirius/ontology/development/07/data/>

SELECT ?activity ?heritage_asset ?element
WHERE {
    ?activity tbox:targets ?heritage_asset ;
      tbox:assesses ?element .
}
'''

In [20]:
text4 = '''CQ_7.4
Who participated in each assessment activity? When? What did it target? What did it assess? What is it documented by?
'''

query4 = '''
PREFIX tbox: <https://w3id.org/sirius/ontology/development/07/schema/>
PREFIX abox: <https://w3id.org/sirius/ontology/development/07/data/>

SELECT ?activity ?agent ?start_date ?end_date ?heritage_asset ?element ?source
WHERE {
    ?activity tbox:hasParticipant ?agent ;
      tbox:atTime ?time_interval ;
      tbox:targets ?heritage_asset ;
      tbox:assesses ?element .
    ?time_interval tbox:hasStartDate ?start_date ;
      tbox:hasEndDate ?end_date .
    OPTIONAL {
      ?activity tbox:isDocumentedBy ?source .
    }
}
'''

In [21]:
queries = [(text1, query1),
           (text2, query2),
           (text3, query3),
           (text4, query4)
           ]

g = rdflib.ConjunctiveGraph()
g.parse(filename, format="turtle", encoding="utf-8")

for query in queries:
    q = prepareQuery(query[1])
    results = g.query(q)
    print(query[0])
    table = []
    for row in results:
        table.append([row[var] for var in results.vars])
    print(tabulate(table, headers=results.vars, tablefmt="psql"))

CQ_7.1
What are the steps that are part of the workflow for the risk assessment of the museum? What is their duration? What are the activities they were executed in?

+--------------------------------------------------------------------+------------------+------------------------------------------------------------------------------+
| step                                                               |   duration_value | activity                                                                     |
|--------------------------------------------------------------------+------------------+------------------------------------------------------------------------------|
| https://w3id.org/sirius/ontology/development/07/data/identify-step |               30 | https://w3id.org/sirius/ontology/development/07/data/identification-activity |
| https://w3id.org/sirius/ontology/development/07/data/analyze-step  |                5 | https://w3id.org/sirius/ontology/development/07/data/analysis-activ